<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%91%D0%B0%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D1%80%D0%B5%D1%86%D0%B5%D0%BF%D1%82%D0%BE%D0%B2.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

Описанные векторизованные строковые операции оказываются наиболее полезными при очистке сильно зашумленных реальных данных. 

Здесь мы рассмотрим пример такой очистки, воспользовавшись полученной из множества различных интернет-источников базой данных рецептов. Наша цель — разбор рецептов на списки ингредиентов, чтобы можно было быстро найти рецепт, исходя из имеющихся в распоряжении ингредиентов.

In [2]:
#!curl -O https://s3.amazonaws.com/openrecipes/20170107-061401-recipeitems.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.3M  100 29.3M    0     0   126k      0  0:03:57  0:03:57 --:--:--  102k0     0   120k      0  0:04:10  0:02:16  0:01:54  134k9.9M    0     0   125k      0  0:03:59  0:02:42  0:01:17  163k


In [3]:
#!gunzip 20170107-061401-recipeitems.json.gz

База данных находится в формате JSON, так что можно попробовать воспользоваться функцией pd.read_json для ее чтения:

In [5]:
import pandas as pd
import numpy as np

In [6]:
try:
    recipes = pd.read_json('20170107-061401-recipeitems.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


Упс! Мы получили ошибку ValueError с упоминанием наличия «хвостовых данных». Если поискать эту ошибку в Интернете, складывается впечатление, что она появляется из-за использования файла, в котором каждая строка сама по себе является корректным JSON, а весь файл в целом — нет. Рассмотрим, справедливо ли это объяснение:

In [12]:
from io import StringIO

with open('20170107-061401-recipeitems.json') as f:
    line = f.readline()
   
pd.read_json(StringIO(line)).shape

(2, 12)

Да, очевидно, каждая строка — корректный JSON, так что нам нужно соединить их все воедино. Один из способов сделать это — фактически сформировать строковое представление, содержащее все записи JSON, после чего загрузить все с помощью pd.read_json :

In [9]:
# Читаем весь файл в массив Python
with open('20170107-061401-recipeitems.json', 'r', encoding="utf-8") as f:
    # Извлекаем каждую строку
    data = (line.strip() for line in f)
    # Преобразуем так, чтобы каждая строка была элементом списка
    data_json = "[{0}]".format(','.join(data))
# Читаем результат в виде JSON

In [13]:
from io import StringIO

recipes = pd.read_json(StringIO(data_json))

In [14]:
recipes.shape

(173278, 17)

Видим, что здесь почти 200 тысяч рецептов и 17 столбцов. Посмотрим на одну из строк, чтобы понять, что мы получили:

In [16]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

Здесь содержится немало информации, но большая ее часть находится в беспорядочном виде, как это обычно и бывает со взятыми из Интернета данными. В частности, список ингредиентов находится в строковом формате и нам придется
аккуратно извлечь оттуда интересующую нас информацию. Начнем с того, что взглянем поближе на ингредиенты:

In [17]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

Средняя длина списка ингредиентов составляет 250 символов при минимальной длине 0 и максимальной — почти 10 000 символов!

Из любопытства посмотрим, у какого рецепта самый длинный список ингредиентов:

In [18]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

Этот рецепт явно выглядит не очень простым.

Можно сделать и другие открытия на основе сводных показателей. Например, посмотрим, сколько рецептов описывают еду, предназначенную для завтрака:

In [19]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

Или сколько рецептов содержат корицу (cinnamon) в списке ингредиентов:

In [20]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

Можно даже посмотреть, есть ли рецепты, в которых название этого ингредиента написано с орфографической ошибкой, как cinamon:

In [21]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

Такая разновидность обязательного предварительного изучения данных возможна благодаря инструментам по работе со строками библиотеки Pandas. Именно в сфере такой очистки данных Python действительно силен.

# Простая рекомендательная система для рецептов

Немного углубимся в этот пример и начнем работу над простой рекомендательной системой для рецептов: по заданному списку ингредиентов необходимо найти рецепт, использующий их все. Концептуально простая, эта задача усложняется
неоднородностью данных: не существует удобной операции, которая позволила бы извлечь из каждой строки очищенный список ингредиентов. Так что мы немного сжульничаем: начнем со списка распространенных ингредиентов и будем искать, в каждом ли рецепте они содержатся в списке ингредиентов. Для упрощения ограничимся травами и специями:

In [22]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley', 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

Мы можем создать булев объект DataFrame , состоящий из значений True и False , указывающих, содержится ли данный ингредиент в списке:

In [23]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE)) 
                             for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


Теперь в качестве примера допустим, что мы хотели бы найти рецепт, в котором используются петрушка (parsley), паприка (paprika) и эстрагон (tarragon). Это можно сделать очень быстро, воспользовавшись методом query() объекта DataFrame:

In [24]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

Мы нашли всего десять рецептов с таким сочетанием ингредиентов. Воспользуемся возвращаемым этой выборкой индексом, чтобы выяснить названия этих рецептов:

In [25]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

Теперь, сократив наш список рецептов почти в 20 000 раз, мы можем принять более взвешенное решение: что готовить на обед.